# Demo de Simulación de Balance - Cantidad de Inversiones

## Configuración inicial de librerías

In [1]:
# Contiene las clases Operacion, Caja y Banco.
import simula_balance as sb

# Contiene las reglas de renovación, por ejemplo,
# toda la caja a overnight.
import reglas_renovacion as rn

# Contiene las métricas, por ejemplo, el C46.
import metricas as met

# Se usa pandas sólo para desplegar claramente el gap de liquidez.
import pandas as pd

## t = 0: Configuración inicial del Banco

In [2]:
# Monto de capital
capital = 10.0

# Sobre este objeto caen los vencimientos de activos y pasivos.
# De aquí se obtiene el dinero para captaciones y colocaciones,
# toda operación, al iniciarse, tiene su correspondiente movimiento
# en la caja.
caja = sb.Caja()

# Construimos un Banco muy sencillo con una colocación bullet
# y una captación (también bullet).
# Lo primero son las tasas de mercado
mercado = {'on_col': 0.0, 'on_cap':0.0,
           'depo2':0.0312, 'depo3':0.033, 'depo4':0.0348,
           'inversion':0.027, 'colocacion':0.045}

# Variable que establece los plazos y porcentajes de renovación a cada plazo.
# Si tenemos x:y significa que al plazo x (medido en número de períodos)
# se hace un y% del monto necesario.
plazos = {2:100}


# Se construye el crédito
credito = sb.Operacion(mercado['colocacion'], 0, 12, 100, {'id':'1', 'ap':'activo', 'producto':'credito'})

# En esta lista se almacenarán las operaciones que insertaremos inicialmente en el banco.
operaciones = [credito]

# Se construye la mezcla de depósitos
i = 0
for plazo in plazos:
    i += 1
    codigo_tasa = 'depo' + str(plazo)
    ponderador = plazos[plazo] / 100.0
    operaciones.append(sb.Operacion(mercado[codigo_tasa], 0, plazo, ponderador * 90,
                                    {'id':str(i), 'ap':'pasivo', 'producto':'depo'}))
    
# Se da de alta el banco inicial
banco = sb.Banco(caja, operaciones, capital)

Vamos a definir algunas funciones auxiliares que harán menos verboso el ejercicio:

In [27]:
def muestra_gap():
    # se determina la situación de liquidez y
    gap = met.construye_gap_liquidez(banco)
    pd_gap = pd.DataFrame([gap()])

    # se muestra el resultado.
    print "Gap de Liquidez a tiempo t = ", banco.tiempo_actual()
    print pd_gap
    print 'c46: ', met.c46(banco, True)
    return

def muestra_resultados():
    # Calcula y muestra el estado de resultados
    er = banco.estado_resultados()
    resultado = 0.0
    for key in er:
        print key, er[key]
        if key == 'activo':
            for interes in er[key]:
                resultado += er[key][interes]
        else:
            for interes in er[key]:
                resultado -= er[key][interes]
    print "resultado: ", resultado
    return

def muestra_balance():
    # Calcula y muestra el balance
    bal = banco.balance()
    for key in bal:
        print key, bal[key]
    return

def muestra_operaciones(t):
    # Muestra los flujos de las operaciones del banco
    activos = banco.activos()
    print 'Activos:'
    print 'prod\t', 'plazo\t', 'monto\t', 'int_dev\t', 'flujo\t'
    for act in activos:
        print act.atributos()['producto'], act.vencimiento(), act.vencimiento() - t, act.monto(), act.interes_devengado(t), act.flujo()
        
    pasivos = banco.pasivos()
    print 'Pasivos:'
    print 'prod\t', 'plazo\t', 'monto\t', 'int_dev\t', 'flujo\t'
    for pas in pasivos:
        print pas.atributos()['producto'], pas.vencimiento(), pas.vencimiento() - t, pas.monto(), pas.interes_devengado(t), pas.flujo()

def avanza():
    # Se avanza un período en el tiempo,
    banco.avanza()
    return
    
def renueva_depo():
    # Se renueva el depósito
    rn.renueva_depos(banco, plazos)

    # se determina la nueva situación de liquidez y
    # se muestra
    muestra_gap()
    return

def toma_inversiones():
    # Se renueva la inversión
    rn.inversion(banco)

    # se determina la nueva situación de liquidez y
    # se muestra.
    muestra_gap()
    return

def arregla_c46():
    # Se arregla el C46,
    rn.cumple_c46(banco, plazos, True)

    # Se muestra gap
    muestra_gap()
    return

def muestra_cartera():
    # Muestra la cartera de operaciones a fecha actual del banco
    for op in banco.cartera():
        print op
    return

def muestra_ratio():
    # Muestra el ratio inversiones / colocaciones
    print round(banco.inv_sobre_col() * 100)

Se construye y muestra el gap de liquidez inicial del banco.

In [4]:
muestra_gap()

Gap de Liquidez a tiempo t =  0
    0   1       2   3   4   5   6   7   8   9   10  11     12  13  14  15  16
0  0.0   0 -91.404   0   0   0   0   0   0   0   0   0  113.5   0   0   0   0
c46:  -0.0


Se muestra el estado de resultados inicial.

In [5]:
muestra_resultados()

activo {'intereses_devengados': 0.0, 'intereses_recibidos': 0.0}
pasivo {'intereses_pagados': 0.0, 'intereses_devengados': 0.0}
resultado:  0.0


Se muestra el balance inicial.

In [6]:
muestra_balance()

patrimonio {'capital_pagado': -10.0}
activo {'caja': 0.0, 'credito': 100.0}
pasivo {'depo': -90.0, 'sobregiro': 0.0}


## t = 1: Se avanza un período en el tiempo.

In [7]:
avanza()

Veamos las operaciones y su resultado, para cada operación se muestra:
- activo/pasivo
- producto
- id
- tiempo inicio
- plazo
- monto inicial
- tasa
- interés devengado
- interés recibido/pagado

In [8]:
muestra_cartera()

('activo', 'credito', '1', 0.0, 12.0, 100.0, 0.045, 1.125, 0.0)
('pasivo', 'depo', '1', 0.0, 2.0, -90.0, 0.0312, -0.702, -0.0)


Verifiquemos el cálculo de interés devengado:

In [11]:
print 'credito:', 100 * 0.045 * 3 / 12.0
print 'depo:', -90 * .0312 * 3 / 12.0 

credito: 1.125
depo: -0.702


Veamos el gap de liquidez y C46.

In [12]:
muestra_gap()

Gap de Liquidez a tiempo t =  1
    0       1   2   3   4   5   6   7   8   9   10     11  12  13  14  15  16
0  0.0 -91.404   0   0   0   0   0   0   0   0   0  113.5   0   0   0   0   0
c46:  91.404


Arreglemos el C46 comprando inversiones.

In [13]:
arregla_c46()

Gap de Liquidez a tiempo t =  1
    0      1        2   3   4   5   6   7   8   9   10     11  12  13  14  15  \
0  0.0  0.617 -92.8299   0   0   0   0   0   0   0   0  113.5   0   0   0   0   

   16  
0   0  
c46:  -0.617


Veamos las nuevas operaciones:

In [14]:
muestra_cartera()

('activo', 'credito', '1', 0.0, 12.0, 100.0, 0.045, 1.125, 0.0)
('activo', 'inversion', '1-inv', 1.0, 2.0, 91.404, 0.027, 0.0, 0.0)
('pasivo', 'depo', '1', 0.0, 2.0, -90.0, 0.0312, -0.702, -0.0)
('pasivo', 'depo', '1-fin_inv', 1.0, 3.0, -91.404, 0.0312, -0.0, -0.0)


## t = 2: Se avanza nuevamente en el tiempo.

In [15]:
avanza()

Veamos el resultado de las operaciones:

In [16]:
muestra_cartera()

('activo', 'credito', '1', 0.0, 12.0, 100.0, 0.045, 2.25, 0.0)
('activo', 'inversion', '1-inv', 1.0, 2.0, 91.404, 0.027, 0.0, 0.617)
('pasivo', 'depo', '1', 0.0, 2.0, -90.0, 0.0312, -0.0, -1.404)
('pasivo', 'depo', '1-fin_inv', 1.0, 3.0, -91.404, 0.0312, -0.713, -0.0)


Chequeo de intereses:

In [22]:
print 'credito:', round(100 * 0.045 * 6 / 12, 4)
print '1-inv:', round(91.404 * 0.027 * 3 / 12, 4)
print 'depo-1:', round(-90 * .0312 * 6 / 12, 4)
print '1-fin_inv', round(-91.404 * 0.0312 * 0 / 12, 4)

credito: 2.25
1-inv: 0.617
depo-1: -1.404
1-fin_inv -0.0


Veamos el gap de liquidez.

In [23]:
muestra_gap()

Gap de Liquidez a tiempo t =  2
      0        1   2   3   4   5   6   7   8   9      10  11  12  13  14  15  \
0  0.617 -92.8299   0   0   0   0   0   0   0   0  113.5   0   0   0   0   0   

   16  
0   0  
c46:  92.2129


Vamos nuevamente a arreglarlo.

In [24]:
arregla_c46()

Gap de Liquidez a tiempo t =  2
    0       1        2   3   4   5   6   7   8   9      10  11  12  13  14  \
0  0.0  0.0054 -93.0248   0   0   0   0   0   0   0  113.5   0   0   0   0   

   15  16  
0   0   0  
c46:  -0.00540000000001


In [25]:
# Notar que el nuevo depo es por un monto menor a la inversión porque partimos con caja > 0.
muestra_cartera()

('activo', 'credito', '1', 0.0, 12.0, 100.0, 0.045, 2.25, 0.0)
('activo', 'inversion', '1-inv', 1.0, 2.0, 91.404, 0.027, 0.0, 0.617)
('activo', 'inversion', '2-inv', 2.0, 3.0, 92.2129, 0.027, 0.0, 0.0)
('pasivo', 'depo', '1', 0.0, 2.0, -90.0, 0.0312, -0.0, -1.404)
('pasivo', 'depo', '1-fin_inv', 1.0, 3.0, -91.404, 0.0312, -0.713, -0.0)
('pasivo', 'depo', '2-fin_inv', 2.0, 4.0, -91.5959, 0.0312, -0.0, -0.0)


Veamos el ratio de inversiones a colocaciones:

In [28]:
muestra_ratio()

90.0


## t = 3: Se avanza nuevamente en el tiempo.

In [29]:
avanza()

Veamos resultado.

In [30]:
muestra_cartera()

('activo', 'credito', '1', 0.0, 12.0, 100.0, 0.045, 3.375, 0.0)
('activo', 'inversion', '1-inv', 1.0, 2.0, 91.404, 0.027, 0.0, 0.617)
('activo', 'inversion', '2-inv', 2.0, 3.0, 92.2129, 0.027, 0.0, 0.6224)
('pasivo', 'depo', '1', 0.0, 2.0, -90.0, 0.0312, -0.0, -1.404)
('pasivo', 'depo', '1-fin_inv', 1.0, 3.0, -91.404, 0.0312, -0.0, -1.4259)
('pasivo', 'depo', '2-fin_inv', 2.0, 4.0, -91.5959, 0.0312, -0.7144, -0.0)


Verifiquemos:

In [32]:
print 'credito:', round(100 * 0.045 * 9 / 12, 4)
print '1-inv:', round(91.404 * 0.027 * 3 / 12, 4)
print '2-inv:', round(92.2129 * 0.027 * 3 / 12, 4)
print 'depo-1:', round(-90 * .0312 * 6 / 12, 4)
print '1-fin_inv', round(-91.404 * 0.0312 * 6 / 12, 4)
print '2-fin_inv', round(-91.5959 * 0.0312 * 3 / 12, 4)

credito: 3.375
1-inv: 0.617
2-inv: 0.6224
depo-1: -1.404
1-fin_inv -1.4259
2-fin_inv -0.7144


Veamos el gap de liquidez.

In [33]:
muestra_gap()

Gap de Liquidez a tiempo t =  3
       0        1   2   3   4   5   6   7   8      9   10  11  12  13  14  15  \
0  0.0054 -93.0248   0   0   0   0   0   0   0  113.5   0   0   0   0   0   0   

   16  
0   0  
c46:  93.0194


Se arregla el C46.

In [34]:
arregla_c46()

Gap de Liquidez a tiempo t =  3
    0       1       2   3   4   5   6   7   8      9   10  11  12  13  14  15  \
0  0.0  0.6225 -94.465   0   0   0   0   0   0  113.5   0   0   0   0   0   0   

   16  
0   0  
c46:  -0.6225


In [35]:
muestra_cartera()

('activo', 'credito', '1', 0.0, 12.0, 100.0, 0.045, 3.375, 0.0)
('activo', 'inversion', '1-inv', 1.0, 2.0, 91.404, 0.027, 0.0, 0.617)
('activo', 'inversion', '2-inv', 2.0, 3.0, 92.2129, 0.027, 0.0, 0.6224)
('activo', 'inversion', '3-inv', 3.0, 4.0, 93.0194, 0.027, 0.0, 0.0)
('pasivo', 'depo', '1', 0.0, 2.0, -90.0, 0.0312, -0.0, -1.404)
('pasivo', 'depo', '1-fin_inv', 1.0, 3.0, -91.404, 0.0312, -0.0, -1.4259)
('pasivo', 'depo', '2-fin_inv', 2.0, 4.0, -91.5959, 0.0312, -0.7144, -0.0)
('pasivo', 'depo', '3-fin_inv', 3.0, 5.0, -93.014, 0.0312, -0.0, -0.0)


## t = 4. Volvemos a avanzar.

In [36]:
avanza()

Veamos resultados.

In [37]:
muestra_cartera()

('activo', 'credito', '1', 0.0, 12.0, 100.0, 0.045, 4.5, 0.0)
('activo', 'inversion', '1-inv', 1.0, 2.0, 91.404, 0.027, 0.0, 0.617)
('activo', 'inversion', '2-inv', 2.0, 3.0, 92.2129, 0.027, 0.0, 0.6224)
('activo', 'inversion', '3-inv', 3.0, 4.0, 93.0194, 0.027, 0.0, 0.6279)
('pasivo', 'depo', '1', 0.0, 2.0, -90.0, 0.0312, -0.0, -1.404)
('pasivo', 'depo', '1-fin_inv', 1.0, 3.0, -91.404, 0.0312, -0.0, -1.4259)
('pasivo', 'depo', '2-fin_inv', 2.0, 4.0, -91.5959, 0.0312, -0.0, -1.4289)
('pasivo', 'depo', '3-fin_inv', 3.0, 5.0, -93.014, 0.0312, -0.7255, -0.0)


Verifiquemos.

In [39]:
print 'credito:', round(100 * 0.045 * 9 / 12, 4)
print '1-inv:', round(91.404 * 0.027 * 3 / 12, 4)
print '2-inv:', round(92.2129 * 0.027 * 3 / 12, 4)
print '3-inv:', round(93.0194 * 0.027 * 3 / 12, 4)
print 'depo-1:', round(-90 * .0312 * 6 / 12, 4)
print '1-fin_inv', round(-91.404 * 0.0312 * 6 / 12, 4)
print '2-fin_inv', round(-91.5959 * 0.0312 * 6 / 12, 4)
print '3-fin_inv', round(-93.014 * 0.0312 * 3 / 12, 4)

credito: 3.375
1-inv: 0.617
2-inv: 0.6224
3-inv: 0.6279
depo-1: -1.404
1-fin_inv -1.4259
2-fin_inv -1.4289
3-fin_inv -0.7255


Veamos el gap.

In [40]:
muestra_gap()

Gap de Liquidez a tiempo t =  4
       0       1   2   3   4   5   6   7      8   9   10  11  12  13  14  15  \
0  0.6225 -94.465   0   0   0   0   0   0  113.5   0   0   0   0   0   0   0   

   16  
0   0  
c46:  93.8425


Arreglemos el C46.

In [41]:
arregla_c46()

Gap de Liquidez a tiempo t =  4
    0       1        2   3   4   5   6   7      8   9   10  11  12  13  14  \
0  0.0  0.0109 -94.6742   0   0   0   0   0  113.5   0   0   0   0   0   0   

   15  16  
0   0   0  
c46:  -0.0109


Registramos el resultado después de 1 año.

In [42]:
muestra_resultados()

activo {'intereses_devengados': 4.5, 'intereses_recibidos': 1.8672999999999997}
pasivo {'intereses_pagados': 4.2588, 'intereses_devengados': 0.7255}
resultado:  1.383


## t = 5: Volvemos a avanzar, mostramos el gap y arreglamos C46.

In [43]:
avanza()

In [44]:
muestra_gap()

Gap de Liquidez a tiempo t =  5
       0        1   2   3   4   5   6      7   8   9   10  11  12  13  14  15  \
0  0.0109 -94.6742   0   0   0   0   0  113.5   0   0   0   0   0   0   0   0   

   16  
0   0  
c46:  94.6633


In [45]:
arregla_c46()

Gap de Liquidez a tiempo t =  5
    0       1       2   3   4   5   6      7   8   9   10  11  12  13  14  15  \
0 -0.0  0.6281 -96.129   0   0   0   0  113.5   0   0   0   0   0   0   0   0   

   16  
0   0  
c46:  -0.6281


## t = 6:Volvemos a avanzar, mostramos el gap y arreglamos C46.

In [46]:
avanza()

In [47]:
muestra_gap()

Gap de Liquidez a tiempo t =  6
       0       1   2   3   4   5      6   7   8   9   10  11  12  13  14  15  \
0  0.6281 -96.129   0   0   0   0  113.5   0   0   0   0   0   0   0   0   0   

   16  
0   0  
c46:  95.5009


In [48]:
arregla_c46()

Gap de Liquidez a tiempo t =  6
    0       1        2   3   4   5      6   7   8   9   10  11  12  13  14  \
0 -0.0  0.0165 -96.3528   0   0   0  113.5   0   0   0   0   0   0   0   0   

   15  16  
0   0   0  
c46:  -0.0165


## t = 7: Volvemos a avanzar, mostramos el gap y arreglamos C46.

In [49]:
avanza()

In [50]:
muestra_gap()

Gap de Liquidez a tiempo t =  7
       0        1   2   3   4      5   6   7   8   9   10  11  12  13  14  15  \
0  0.0165 -96.3528   0   0   0  113.5   0   0   0   0   0   0   0   0   0   0   

   16  
0   0  
c46:  96.3363


In [51]:
arregla_c46()

Gap de Liquidez a tiempo t =  7
    0       1        2   3   4      5   6   7   8   9   10  11  12  13  14  \
0 -0.0  0.6338 -97.8224   0   0  113.5   0   0   0   0   0   0   0   0   0   

   15  16  
0   0   0  
c46:  -0.6338


## t = 8. Última iteración.

In [52]:
avanza()

In [53]:
muestra_gap()

Gap de Liquidez a tiempo t =  8
       0        1   2   3      4   5   6   7   8   9   10  11  12  13  14  15  \
0  0.6338 -97.8224   0   0  113.5   0   0   0   0   0   0   0   0   0   0   0   

   16  
0   0  
c46:  97.1886


In [54]:
arregla_c46()

Gap de Liquidez a tiempo t =  8
    0       1        2   3      4   5   6   7   8   9   10  11  12  13  14  \
0 -0.0  0.0222 -98.0611   0  113.5   0   0   0   0   0   0   0   0   0   0   

   15  16  
0   0   0  
c46:  -0.0222


In [55]:
muestra_resultados()

activo {'intereses_devengados': 9.0, 'intereses_recibidos': 4.4346}
pasivo {'intereses_pagados': 10.1206, 'intereses_devengados': 0.7513}
resultado:  2.5627


In [56]:
muestra_cartera()

('activo', 'credito', '1', 0.0, 12.0, 100.0, 0.045, 9.0, 0.0)
('activo', 'inversion', '1-inv', 1.0, 2.0, 91.404, 0.027, 0.0, 0.617)
('activo', 'inversion', '2-inv', 2.0, 3.0, 92.2129, 0.027, 0.0, 0.6224)
('activo', 'inversion', '3-inv', 3.0, 4.0, 93.0194, 0.027, 0.0, 0.6279)
('activo', 'inversion', '4-inv', 4.0, 5.0, 93.8425, 0.027, 0.0, 0.6334)
('activo', 'inversion', '5-inv', 5.0, 6.0, 94.6633, 0.027, 0.0, 0.639)
('activo', 'inversion', '6-inv', 6.0, 7.0, 95.5009, 0.027, 0.0, 0.6446)
('activo', 'inversion', '7-inv', 7.0, 8.0, 96.3363, 0.027, 0.0, 0.6503)
('activo', 'inversion', '8-inv', 8.0, 9.0, 97.1886, 0.027, 0.0, 0.0)
('pasivo', 'depo', '1', 0.0, 2.0, -90.0, 0.0312, -0.0, -1.404)
('pasivo', 'depo', '1-fin_inv', 1.0, 3.0, -91.404, 0.0312, -0.0, -1.4259)
('pasivo', 'depo', '2-fin_inv', 2.0, 4.0, -91.5959, 0.0312, -0.0, -1.4289)
('pasivo', 'depo', '3-fin_inv', 3.0, 5.0, -93.014, 0.0312, -0.0, -1.451)
('pasivo', 'depo', '4-fin_inv', 4.0, 6.0, -93.22, 0.0312, -0.0, -1.4542)
('pasivo',